**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_user_json_ld_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [ ]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-25 12:04:52,145 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-25 12:04:52.163 | INFO     | __main__:<module>:3 - b'f783c72b38e587ece984c3e31fff1bee40c0f5bf0f19dd8165b75fbaba582766'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-01-25 12:04:57.845 | INFO     | __main__:<module>:13 - User Role: publisher
2025-01-25 12:04:57.848 | INFO     | __main__:<module>:24 - Creating account with name: optimistic_dewdney
2025-01-25 12:04:57.850 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-25 12:04:57.874 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'f783c72b38e587ece984c3e31fff1bee40c0f5bf0f19dd8165b75fbaba582766'
2025-01-25 12:04:57.878 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:04:57.882 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-25 12:04:57.885 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-25 12:04:57,912 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:04:57,914 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9

2025-01-25 12:04:58,405 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-25 12:04:58,414 - INFO - ('COMMITTED', 5, 0)
2025-01-25 12:04:58.424 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 81
2025-01-25 12:04:58.427 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


b'0570d9af2cc9879ad39b623e627f14b5637bc754d591db8e0a023df5486b9f26'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-25 12:11:02.171 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-25 12:11:02,177 - WARNING - create_account_contract_address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:11:02,195 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:11:04,916 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-25 12:11:04,923 - INFO - ('COMMITTED', 5, 0)
2025-01-25 12:11:04,925 - WARNING - Account Data: {'address': '44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9', 'hex_hash': 'd762f23c247dc2db69fb8f22bf27109fadf54eb5260330d72143a913ba229a57'}
2025-01-25 12:11:04.929 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-25 12:11:04,933 - INFO - Appended data under 'optimistic_dewdney@test' in logs/account_data.json
2025-01-25 12:11:04.934 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-25 12:11:04.937 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account"
2025-01-25 12:11:

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [7]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-25 12:15:09,756 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-25 12:15:09.760 | INFO     | __main__:<module>:3 - b'8ab0f4ff1b75e1c1a1102b5eaa5854b0c23507747e387a15df0e03e57699a1f7'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [8]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-25 12:15:14.175 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-25 12:15:14.201 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'8ab0f4ff1b75e1c1a1102b5eaa5854b0c23507747e387a15df0e03e57699a1f7'
2025-01-25 12:15:14.203 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 2A2B7EBDCCD639978CAB491B16ECAD343A56AF42
2025-01-25 12:15:14.204 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-25 12:15:14.206 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-25 12:15:14,210 - INFO - Processing account: optimistic_dewdney@test
2025-01-25 12:15:14,212 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Optimistic Dewdney', 'foaf:mbox': 'optimistic_dewdney@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Universidad de Valencia'},

6 - Queries the blockchain again to confirm the succesfull details setting.

In [9]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
user_json_ld_cid = process_raw_data(details)

2025-01-25 12:16:49.725 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-25 12:16:49,731 - WARNING - create_account_contract_address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:16:49,771 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:16:50,411 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-25 12:16:50,419 - INFO - ('COMMITTED', 5, 0)
2025-01-25 12:16:50,422 - WARNING - Account Data: {'address': '44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9', 'hex_hash': '7ac1735c19dede4d606299fbdf4f5c92258bc6cae962c3603158fb86c964e5df'}
2025-01-25 12:16:50.425 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-25 12:16:50,431 - INFO - Appended data under 'optimistic_dewdney@test' in logs/account_data.json
2025-01-25 12:16:50.436 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-25 12:16:50.439 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account"
2025-01-25 12:16:

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(user_json_ld_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-25 11:47:21.010 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'youthful_ride@test', 'schema:publicKey': '535e30f82a4c3488df4502917a6a472c50096c3277e8c84d20e875bb5c0a3c57', 'schema:roleName': 'author'}, 'foaf:mbox': 'youthful_ride@email.com', 'foaf:name': 'Youthful Ride', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Tui Online University'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '9036-1605-4504-X'}}
